Large Steps in Inverse Rendering of Geometry
======================================

This is an example on how to use our method for shape optimization with differentiable rendering. It uses `nvdiffrast` for the differentiable rendering part.

In [ ]:
import torch
import os
import cupy as cp

from tqdm import trange
from scripts.load_xml import load_scene
from scripts.geometry import (
    compute_vertex_normals, 
    compute_face_normals,
)
from scripts.render import NVDRenderer

# 0. Loading the scene

Let's start by loading a scene

In [ ]:
# Load the scene
filepath = os.path.join(os.getcwd(), "scenes", "centered", "centered.xml")
scene_params = load_scene(filepath)

# Load reference shape
v_ref = scene_params["mesh-target"]["vertices"]
n_ref = scene_params["mesh-target"]["normals"]
f_ref = scene_params["mesh-target"]["faces"]

v_ref, n_ref, f_ref = [f.contiguous() for f in [v_ref, n_ref, f_ref]]
# Load source shape
#v = scene_params["mesh-source"]["vertices"]
#f = scene_params["mesh-source"]["faces"]

# 1. Rendering references

Then, we need to setup a differentiable rendering pipeline. Here, we use an implementation based on `nvdiffrast`:

We initialize it once, so it loads the camera data, the environment map and precomputes the shading model, using spherical harmonics in this case.

In [ ]:
# Initialize the renderer
renderer = NVDRenderer(scene_params, shading=True, boost=3)

Let's render the target shape to use as a reference for the optimization:

In [ ]:
ref_imgs = renderer.render(v_ref, n_ref, f_ref)

Let's look at one of these references

In [ ]:
import matplotlib.pyplot as plt
plt.imshow((ref_imgs[7,...,:-1].clip(0,1).pow(1/2.2)).cpu().numpy(), origin='lower')

# 2. Parameterizing

Now it's time to setup the optimization. First, let us import what we need. We need an optimizer, `AdamUniform`, and the functions that allow us to convert back and forth between vertex positions and their parameterization.

In [ ]:
steps = 1000 # Number of optimization steps
step_size = 3e-2 # Step size
lambda_ = 19 # Hyperparameter lambda of our method, used to compute the matrix (I + lambda_*L)

Now we need to parameterize our shape. 

In [ ]:
#from src.mesh.ellipsoid import Ellipsoid
#from src.mesh.geoid import Geoid
from src.mesh.cube import NetCube
from argparse import Namespace

opt = Namespace(
    n = 82,
    nfc=32, 
    min_nfc=32, 
    ker_size=3,
    num_layer=5,
    stride=1,
    padd_size=0,
    nc_im=3,
)

n = 128
cube = NetCube(n, opt, kernel=7, sigma=2).cuda()

#radii = torch.rand(3, device=v_ref.device, requires_grad=True)
v, f = cube()
# n_ref = compute_face_normals(v_ref, f_ref)
[f.shape for f in [v, f]]

Let's initialize our optimizer, `AdamUniform`

In [ ]:
from largesteps.optimize import AdamUniform

In [ ]:
#u.requires_grad = True

#opt = AdamUniform(cube.parameters(), step_size)
opt = torch.optim.Adam(cube.parameters(), step_size)

In [ ]:
# Dictionary that is returned in the end, contains useful information for debug/analysis
v_steps = torch.zeros((steps+1, *v.shape))#, device='cuda')
losses = torch.zeros(steps+1)#, device='cuda')

# 3. Running the optimization

And now we can run our optimization. The only difference with "regular" optimization here is the call to `from_differential` in the loop body, that converts the parameterization to vertex coordinates. The rest of the optimization pipeline is unchanged.

In [ ]:
opt.zero_grad()

# Optimization loop
for it in trange(steps):
    #v, f = ellipsoid(torch.sigmoid(radii) * 2.)
    #v, f = geoid()
    v, f = cube()
    # Recompute vertex normals
    face_normals = compute_face_normals(v, f)
    n = compute_vertex_normals(v, f, face_normals)

    # Render images
    v, n, f = [f.contiguous()  for f in [v, n, f]]
    opt_imgs = renderer.render(v, n, f)

    # Compute L1 image loss
    loss = (opt_imgs - ref_imgs).abs().mean()

    # Record optimization state for later processing
    with torch.no_grad():
        losses[it] = loss.cpu()
        v_steps[it] = v.cpu()

    # Backpropagate
    opt.zero_grad()
    loss.backward()
    
    # Update parameters
    opt.step()

In [ ]:
with torch.no_grad():
    # Render images
    opt_imgs = renderer.render(v, n, f)
    # Compute L1 image loss
    loss = (opt_imgs - ref_imgs).abs().mean()
    losses[-1] = loss
    v_steps[-1] = v

# 4. Visualizing the results

In [ ]:
from meshplot import plot
from ipywidgets import interact
import numpy as np

In [ ]:
v_numpy = v_steps.cpu().numpy()
f_numpy = f.cpu().numpy()

In [ ]:
shading_params = {
    "width": 600, "height": 600,
    "antialias": True,
    "colormap": "viridis",
    "wireframe": True, "wire_width": 0.03, "wire_color": "black"
}

Let's plot the loss:

In [ ]:
plt.plot(losses.cpu().numpy())

And visualize the mesh across iterations:

In [ ]:
@interact(it=(0, steps-1))
def plot_verts(it):
    plot(v_numpy[it], f_numpy, shading=shading_params)